In [1]:
import sys
import pandas as pd
import numpy as np
# Note appending the path of our custom folder from last session to access db_utils
sys.path.append('../../Lesson_2/custom')
from db_utils import get_connection, validate_connection, get_data
pd.options.display.float_format = '{:,.2f}'.format

# Motivating Question

How can we manipulate data in pandas to to answer our questions?
What are some common data manipulations steps we should feel comfortable with in pandas?
There's more than one way to skin a cat

We're going to be working with some ```Milestone Updates``` data which is somewhat messy but the only source for a lot of questions we'll need to solve.

<i>Note: the column update_lead_hours here is a calculation of hours between when the update was made, and when the milestone occured. Negative means the milestone was created after it happened, postive means the milestone was created ahead of the actual event.</i>

In [2]:
# We'll get started by establishing a database connection and pulling a predetermined dataset of milestone updates
# We're selecting for only July 2020 and only arrival/departure milestones to ignore T&T events like last free day.
conn, cur = get_connection()
df = get_data('milestones.sql','file', conn)


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


# Questions:
    1. What percentage of actual updates are human vs automated
    2. What (if any) is the improvement in update speed earned by automation and operations teams
    
    The second two will be part 2 of this lesson
    
    3. What milestones are frequently missing per given mode?
    4. What are the fastest updates per mode
    
    Keeping in mind for the above, we'll need to handle outliers and extraneous/missing data.
   
## Data Exploration

Let's take a look through our data to have an idea of what we're working with.

<b>Can we identify any issues we might have in attempting to answer our motivating questions?</b>


In [22]:
# Question? Explore the data and determine some issues we may see with this analysis
df.head()


,id,shipment_id,shipment_mode,shipment_load_type,leg_mode_type,address_type,update_event_type,update_date_type,source,created_by,update_lead_hours
0,vu3219541986076666330647937003905235TLLU1613600,860766,Ocean,LCL,Truck - Domestic,consolidation,departure,scheduled,human,Cassie Yang,155.00
1,vu3219541886076666330647937003561493TLLU1613600,860766,Ocean,LCL,Truck - Domestic,departure_port,arrival,scheduled,human,Cassie Yang,155.00
2,vu32295136860525665492378826761106069ct,860525,Trucking - Domestic,FTL,Truck - Domestic,destination,arrival,scheduled,human,Nate Lund,109.00
3,vu32295137860525665492378826761106066ct,860525,Trucking - Domestic,FTL,Truck - Domestic,origin,departure,scheduled,human,Nate Lund,100.00
4,vu32295096860523665491278826651106069ct,860523,Trucking - Domestic,FTL,Truck - Domestic,destination,arrival,scheduled,human,Nate Lund,109.00


# 1. Percentage of  updates human vs automated

I'm solving this by stringing together some logic to only solve for air and ocean milestones (that's all we want right now) and choosing only actual dates, not scheduled. From there we get a total count of rows, and final determine the count of each value in a series before dividing by the toal.


In [4]:
# One less-readable way to filter data is to create a series that holds conditional expressions
# for example
ocean_filter = df.shipment_mode == 'Ocean'
air_filter = df.shipment_mode == 'Air'
milestones_filter = df.update_date_type == 'actual'
final_filter =  (ocean_filter | air_filter ) & milestones_filter

#We'll stop to discuss what these above filters mean here.
total_rows = df[final_filter]['id'].count()
df[final_filter]['source'].value_counts()/total_rows


human           0.67
no user found   0.29
cargosmart      0.02
inttra          0.02
crux            0.00
Name: source, dtype: float64

### Group Work Session
After examining this solution work through your own solution utilizing another way to cut data. This could be solved using iloc, loc, or <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html">query which I suggest</a>. You can also use the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">group_by function</a> to circumvent value_counts. In total this question can be answered in a single line.

Try to answer this question in the most pythonic, or efficient way, or output the data in a more permanent way.

In [5]:
# SOLUTION CELL

df.query("update_date_type == 'actual' & shipment_mode in ('Ocean', 'Air')").groupby(by='source')['id'].count()/df.query("update_date_type == 'actual'")['id'].count()

source
cargosmart      0.02
crux            0.00
human           0.66
inttra          0.02
no user found   0.29
Name: id, dtype: float64

# 2. Improvement in Update Speed for Automation

Well we learned in the top portion that the majority of our updates are coming from humans, with a bit of our updates provided by crux/inttra/cargosmart and almost 30% with no user found. If you dig a little bit deeper you'll find that we also have some null values present in this column. 

In this next challenge we're going to do a couple different steps.
    
    1. Use a separate csv which maps "Human" users to bots, individuals or groups
    2. Using logic source and user type, determine if the the update was a bot, individual, or shared account. 
    3. Drop null values
    4. Eliminate outlier data and calculate average update speed.

In [6]:
# retrieve user/type mapping
user_types = pd.read_csv('user_types.csv')

# create a new column with user_type in our current DF
data = df[final_filter]
data = pd.merge(data , user_types, on='created_by', how='left')

# drop rows where the created_by or source are null
data = data.dropna(how='any', subset=['source','created_by'])

In [7]:
# define a function for determining what type of user we have
def get_user(row):
    if row['source'] in ('crux', 'inttra', 'cargosmart'):
        return 'bot'
    if (row['source'] == 'no user found'):
        return row['user_type']
    if (row['source'] == 'human'):
        return row['user_type']
    else:
        return None

# apply this function to our dataframe to create a new column from our logic
data['categorical_user_type'] = data.apply(get_user, axis=1)

In [8]:
# we'll then want to drop all null values from this colume that we'll be using moving forward
data = data.dropna(how='any', subset=['categorical_user_type'])

# Now let's take a look at what we're working with
data['update_lead_hours'].describe()

count       316,491.00
mean           -313.92
std          40,065.12
min     -15,936,287.00
25%             -95.00
50%             -20.00
75%              -1.00
max              12.00
Name: update_lead_hours, dtype: float64

## Removing Outliers

Describing the data we see that the long tail of update lead hours is quite long. On average our milestone updates are logged 313 hours after occuring, with the median sitting at 20 hours. For this analysis we can probably assume no "actual" updates should be happening before the fact, so we'll drop all values greater than 0. 
It seems like some of our large outliers are significantly skewing our data, I think an assumption of 2 weeks is reaonsable to expect for the longest it should reasonably take to update a milestone, so we'll drop all rows with lead hours over 336 hours.

In [9]:
final_dataset = data[(data['update_lead_hours'] < 0) & (data['update_lead_hours'] > -336.0)]

# our final dataset has significantly less values
final_dataset['id'].count()

229646

In [20]:
final_dataset.groupby(by='categorical_user_type')['update_lead_hours'].mean()

categorical_user_type
bot           -5.90
individual   -72.65
shared       -24.90
Name: update_lead_hours, dtype: float64

## Solution
This gives us the expected solution! Bots are significantly faster than human users, and accounts run by individuals are slower at making updates than those owned by shared accounts (teams)